In [59]:
from pathlib import Path
import os
from dotenv import load_dotenv
from typing import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
# read dataset into datasets
from datasets import Dataset
import pandas as pd
dataset_name = "five-star-trajectories"
dataset_path = Path(f"lm_act_eval/.cache/{dataset_name }")

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible
from typing import Callable
from tqdm import tqdm
tqdm.pandas()


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Mapping[str, gymnasium.spaces.space.Space[typing.Any]] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


### Load other data

In [11]:
traj_df = pd.read_csv(dataset_path/'csv/data+gptv.csv')
print(traj_df.columns)

Index(['Unnamed: 0', 'session_id', 'idx_in_session', '_id', 'event_id',
       'parent_id', 'event_type', 'event_name', 'config', 'children_ids',
       'inputs', 'outputs', 'error', 'start_time', 'end_time', 'metadata',
       'feedback', 'metrics', 'user_properties', 'source', 'duration',
       'project_id', 'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata',
       'GPTV response', 'chat_completion_messages_content'],
      dtype='object')


### Process data

In [50]:
from autoevals.string import Levenshtein

In [45]:
extract_action_fs = lambda x: extract_action(x) if type(x) == str else ""
extract_thought_fs = lambda x: extract_thought(x) if type(x) == str else ""
extract_explanation_fs = lambda x: extract_explanation(x) if type(x) == str else ""
process_fs = {
  "action": extract_action_fs,
  "thought": extract_thought_fs,
  "explanation": extract_thought_fs
}
metric_registry = {
    "edit": Levenshtein()
}

In [49]:
metric_comp = pd.DataFrame()
gt_col = "ground_truth"
gen_col = "GPTV"
# Prepare metric input columns
for c in [gt_col, gen_col]:
  for k, func in process_fs.items():
    traj_df[c+'_'+k] = traj_df[c].progress_apply(func)

    

100%|██████████| 3382/3382 [00:00<00:00, 435822.05it/s]


### Eval Metrics

In [ ]:
config_metrics = {
    "Actions": {"edit": "edit"}
}
metric_registry = {
    "edit": Levenshtein()
}

# Step break
# compute metrics
for m, func in metric_registry.items():
  for k in process_fs:
    traj_df[c+'_'+k] = traj_df[c].progress_apply(func)

In [78]:
# Trajectory
metric_comp_traj_df = pd.DataFrame()
def extract_trajectory(traj_df, target_col: str='ground_truth') -> List:
  sorted_grouped_texts = (
    traj_df.sort_values(by=['session_id', 'idx_in_session'])
    .groupby('session_id')[target_col]
    .apply(list)
  )
  return sorted_grouped_texts

for c in [gt_col, gen_col]:
  for k, func in process_fs.items():
    metric_comp_traj_df[c+'_'+k+'_trajectory'] = extract_trajectory(traj_df, target_col=c+'_'+k)
  
# trajectory_gt = extract_trajectory(target_col=gt_col)
# trajectory_target = extract_trajectory(target_col=gen_col)
metric_comp_traj_df.head(1)

,ground_truth_action_trajectory,ground_truth_thought_trajectory,ground_truth_explanation_trajectory,GPTV_action_trajectory,GPTV_thought_trajectory,GPTV_explanation_trajectory
session_id,,,,,,
0007f9ea-9830-4c63-a560-3b133b2b57ca,"[CLICK 3, CLICK 40, CLICK 4, TYPE 7 ""rear left...",[PLAN: To remove the item that was just added ...,[PLAN: To remove the item that was just added ...,"[, , , , , , ]","[, , , , , , ]","[, , , , , , ]"


In [85]:
for c in [gt_col, gen_col]:
  for k, func in process_fs.items():
    
    distances = traj_df[c+'_'+k].apply(lambda x: distance(x, extracted2.loc[x.name]))


SyntaxError: incomplete input (2792572238.py, line 3)

In [ ]:
def evaluate_trajectory(dataset):
    """
    eligible dataset
    """
    metric_results = []
    for index, row in dataset[eligible].iterrows():
        result_row = {}
        for metric_config in config_metrics:
            metric_name = metric_config["name"]
            if metric_registry.get(metric_name):
                metric_func = metric_registry[metric_name]
                result_row[metric_name] = metric_func(row)
        metric_results.append(result_row)

Logging

In [81]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [82]:
opentable_artifact = wandb.Artifact("opentable_trajectories_gptv_compare.v0", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_traj = wandb.Table(dataframe=metric_comp_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(opentable_table_traj, "opentable-trajectory_comparison")
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv-eligible.csv'))


ArtifactManifestEntry(path='opentable-trajectory_comparison.table.json', digest='BDwjRNg7S/8yzW9UpY9ezw==', size=1994147, local_path='/home/opadmin/.local/share/wandb/artifacts/staging/tmp_i4jhxjd')

In [84]:
# Start a W&B run to log data
run = wandb.init(project='trajectory_eval', name="opentable-GPTV", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv_metric_comparison": metric_comp_traj_df})

# and Log as an Artifact to increase the available row limit!
# run.log_artifact(opentable_artifact)

Problem at: /home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/wandb/sdk/wandb_init.py 854 getcaller


CommError: Run initialization has timed out after 90.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-